# Entitlements in TOC-eligible parcels

In [1]:
import boto3
import geopandas as gpd
import intake
import numpy as np
import pandas as pd
import pcts_parser
import utils

In [2]:
catalog = intake.open_catalog("../catalogs/*.yml")

s3 = boto3.client('s3')
bucket_name = 'city-planning-entitlements'

## Parcels
* Won't know which AINs are used inpcts.CASE_NBRep all of them, but have a way to identify how many obs to drop later on
* Use the PCTS util to get PCTS data, then subset to TOC only

## Import files

In [5]:
def make_tract_level(start_date):    
    # Import data
    pcts = pd.read_parquet(f's3://{bucket_name}/data/final/master_pcts.parquet')
    parents = pd.read_parquet(f's3://{bucket_name}/data/final/parents_with_suffix.parquet')
    crosswalk_parcels_tracts = pd.read_parquet(f's3://{bucket_name}/data/crosswalk_parcels_tracts.parquet')
    
    # Subset PCTS, we'll start with last 10 years
    pcts = pcts[pcts.CASE_FILE_DATE >= start_date]
    
    # (1a) Only keep parent cases
    m1 = pd.merge(pcts, parents, on = 'PARENT_CASE', how = 'inner', validate = 'm:1')

    # (1b) Make cases parcel-level
    parents_by_parcel = (m1.groupby(['AIN'])
                         .agg({'PARENT_CASE':'count'})
                         .reset_index()
                         .rename(columns = {'PARENT_CASE':'num_cases'})
                        )

    # (1c) Merge in tract info and aggregate to tract-level
    m2 = pd.merge(parents_by_parcel, crosswalk_parcels_tracts, on = 'AIN', how = 'inner', validate = '1:1')

    # Even though the column num_AIN shows there are some parcels with more than 1 obs,
    # once we merged in parent cases, no AIN shows up more than once

    # (1d) Subset by prefixes?
    def subset_by_prefix_suffix(row):
        if any(p in row.CASE_NBR for p in prefix_or_suffix_list):
            return True
        else: 
            return False 
    
    ent_by_tract = m2.groupby(['GEOID', 'pop']).agg({'num_cases':'sum', 'AIN':'count'}).reset_index()
    
    
    # (2a) Only keep suffixes
    suffix = m1.loc[:, '1A':'ZV']
    
    m3 = pd.merge(m1[['AIN']], suffix, left_index = True, right_index = True)

    # (2b) Make suffixes parcel-level
    suffix_by_parcel = (m3.pivot_table(index = 'AIN', aggfunc = 'sum')
                        .reset_index()
                       )

    # (2c) Merge in tract info and aggregate to tract-level
    m4 = pd.merge(suffix_by_parcel, crosswalk_parcels_tracts, 
                                 on = 'AIN', how = 'left', validate = '1:1')

    # Aggregate the number of suffixes by tract
    suffix_by_tract = m4.pivot_table(index = ['GEOID', 'pop'], aggfunc = 'sum').reset_index()
    
    
    # (3) Merge number of cases and suffixes by tract
    df = pd.merge(ent_by_tract, suffix_by_tract, on = ['GEOID', 'pop'], how = 'left', validate = '1:1')
        
    # (4) Make sure everything returns as integers and not floats
    colnames = list(df.columns)

    for r in ['GEOID', 'pop', 'AIN']:
        colnames.remove(r)
    
    # Count number of suffixes, since each ENT can have any number of suffixes involved
    df[colnames] = df[colnames].fillna(0).astype(int) 
    df['num_suffix'] = df[colnames].sum(axis=1)
        
    return df

In [6]:
start_date = "2017-10"
pcts = make_tract_level(start_date)

# Just keep TOC ent for the tract
keep = ['GEOID', 'pop', 'num_cases', 'AIN', 'num_AIN', 'parcel_tot', 'parcelsqft', 'TOC',]
pcts = pcts[keep]

In [7]:
# Prep census tract data to make map
census = catalog.census_tracts.read()
census = (census[['GEOID10', 'geometry']]
        .assign(num = 0)
        .rename(columns = {'GEOID10':'GEOID'})
         ).to_crs("EPSG:4326")

In [8]:
# Make a map showing TOC ent by tract, but also show all census tracts in LA
gdf = pd.merge(census.drop(columns = 'num'), pcts, on = 'GEOID', how = 'inner', validate = '1:1')
gdf = gdf[gdf.TOC > 0]

In [9]:
import ipyleaflet
from ipyleaflet import Map, GeoData, LayersControl, basemaps, WidgetControl
from ipywidgets import link, FloatSlider, Text, HTML
from ipywidgets.embed import embed_minimal_html
import branca.colormap
import json

geo_data = json.loads(gdf.set_index('GEOID').to_json())

# Take what we want to map and turn it into a dictionary
# Can only include the key-value pair, the value you want to map, nothing more.
df = dict(zip(gdf['GEOID'].tolist(), gdf['TOC'].tolist()))


census_geo = json.loads(census.set_index('GEOID').to_json())
census_df = dict(zip(census['GEOID'].tolist(), census['num'].tolist()))

In [10]:
m = ipyleaflet.Map(center = (34.0536, -118.2427), zoom = 11,
                  basemap = basemaps.CartoDB.Positron)

census_tracts = ipyleaflet.Choropleth(
    geo_data = census_geo,
    choro_data = census_df, 
    colormap = branca.colormap.LinearColormap([
        "#FFFFFF",
        "#FFFFFF",        
    ]),
    border_color = '#999999',
    style = {'fillOpacity': 0.4, 'weight': 0.5, 'color': '#999999', 'opacity': 0.8},
    value_min = 0,
    value_max = 0,
    name = 'Census Tracts'
)

toc = ipyleaflet.Choropleth(
    geo_data = geo_data,
    choro_data = df,
    colormap = branca.colormap.LinearColormap([
        "#C9FFFD",
        "#06BEE1",
        "#1768AC",
        "#2541B2",
        "#03256C"
    ]),
    border_color = 'white',
    style = {'fillOpacity': 0.8, 'weight': 0.5, 'color': 'white', 'opacity': 0.8},
    hover_style = {'fillOpacity': 0.85},
    value_min = 0,
    value_max = 60,
    name = 'TOC entitlements'
)

html = HTML(''' 
    Hover over a tract
''')

html.layout.margin = '0 px 10px 10px 10px'

def click_handler(event = None, id = None, properties = None):
    label.value = properties['TOC']

def update_html_tract(feature, id, **kwargs): 
    html.value = '''
        Census Tract:  
        <b>{}</b> <br>
        Number of TOC Entitlements (2017-2019):
        {} 
    '''.format(id, feature['properties']['TOC'])    
    
toc.on_click(click_handler) 
toc.on_hover(update_html_tract)

m.add_layer(census_tracts)
m.add_layer(toc)

control = ipyleaflet.WidgetControl(widget = html, position = 'topright')
layers_control = ipyleaflet.LayersControl(position = 'topright')

m.add_control(control)
m.add_control(layers_control)

m.layout.height = '100%'
m.layout.min_height = '600px'

m

Map(center=[34.0536, -118.2427], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', '…

In [ ]:
parcels = gpd.read_file(f'zip+s3://{bucket_name}/gis/intermediate/TOC_Parcels.zip')

# Grab the centroids and count number of duplicate obs
parcels2 = utils.get_centroid(parcels)

In [ ]:
# Subset to eligible zones and see which TOC-eligible parcels also fall in eligible zones
zoning = gpd.read_file(f's3://{bucket_name}/gis/raw/parsed_zoning.geojson')

eligible_zones = ['R2', 'R3', 'RAS3', 'R4', 'RAS4', 'R5', 
              'RD1.5', 'RD2', 'RD3', 'RD4', 'RD5', 'RD6', 
              'C1', 'C2', 'C4', 'C5']

eligible_zoning = zoning[zoning.zone_class.isin(eligible_zones)]

parcels_with_zoning = gpd.sjoin(parcels2, eligible_zoning, 
                                how = 'inner', op = 'intersects').drop(columns = ['index_right'])

## Process PCTS
* Join parcels to zoning
* Subset for eligible zones and eligible PCTS prefixes to see how many TOC-eligible parcels fall into eligible zones

In [ ]:
def zoning_pcts_processing(df, parcels_with_zoning): 
    # Merge in zoning and TOC info about the parcel
    m1 = pd.merge(df, parcels_with_zoning, on = ['AIN'], how = 'inner')
    
    # Drop duplicates
    m1 = m1.drop_duplicates()

    # Parse PCTS string and grab prefix
    parsed_col_names = ['prefix']

    def parse_pcts(row):
        try:
            z = pcts_parser.PCTSCaseNumber(row.CASE_NBR)
            return pd.Series([z.prefix], index = parsed_col_names)
        except ValueError:
            return pd.Series([z.prefix], index = parsed_col_names)

    parsed = m1.apply(parse_pcts, axis = 1)
    m2 = pd.concat([m1, parsed], axis = 1)
    
   
    # Subset by PCTS prefix, drop ENV/ADM/PAR cases
    drop_prefix = ['ENV', 'ADM', 'PAR']
    m3 = m2.loc[~m2.prefix.isin(drop_prefix)]
    
    # Subset by CASE_ACTION_ID -- let's use all cases for now (but approved cases are 1, 2, 11)
    approved_cases = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
    m4 = m3.loc[m3.CASE_ACTION_ID.isin(approved_cases)]
        
    # At this point, no more duplicates by PARENT_CASE - AIN combination
    return m4

In [ ]:
def tag_toc_entitlements(df):
    keep_col = ['CASE_NBR', 'id', 'CASE_ACTION_ID', 'CASE_FILE_DATE', 
            'AIN', 'TOC_Tier', 'zone_class']
    
    df = (df[keep_col]
          .assign(is_TOC = df.CASE_NBR.str.contains('TOC').astype(int))
         )
    
    # Make into parcel-level df
    df2 = (df.groupby(['AIN', 'TOC_Tier', 'zone_class', 'is_TOC'])
           .agg({'id':'count'})
           .reset_index()) 

    # Make wide
    df2 = df2.assign(
        num_TOC = df2.apply(lambda row: row.id if row.is_TOC == 1 else np.nan, axis = 1),
        num_nonTOC = df2.apply(lambda row: row.id if row.is_TOC == 0 else np.nan, axis = 1)
    )
    
    
    # If there are multiple obs for the same AIN, fill the NaNs with the max from the other column 
    # Then, drop duplicates
    df2 = df2.assign(
        num_TOC = df2.num_TOC.fillna(df2.groupby('AIN')['num_TOC'].transform('max')),
        num_nonTOC = df2.num_nonTOC.fillna(df2.groupby('AIN')['num_nonTOC'].transform('max'))
    )
    
    df3 = df2.drop_duplicates(subset = ['AIN', 'TOC_Tier', 'zone_class', 'num_TOC', 'num_nonTOC'])

    df3 = (df3.assign(
            num_TOC = df3.num_TOC.fillna(0).astype(int),
            num_nonTOC = df3.num_nonTOC.fillna(0).astype(int)
        ).drop(columns = ['is_TOC', 'id'])
    )
    
    # Merge in centroids for these parcels (much easier to plot)
    df4 = pd.merge(df3, parcels2, on = ['AIN', 'TOC_Tier'], how = 'inner').drop(
                    columns = ['x', 'y', 'obs', 'num_obs'])
    
    df4.rename(columns = {'centroid':'geometry'}, inplace = True)
    df4 = gpd.GeoDataFrame(df4)
    df4.crs = {'init':'epsg:2229'}

    return df4

In [ ]:
pcts2 = zoning_pcts_processing(pcts, parcels_with_zoning)

In [ ]:
df = tag_toc_entitlements(pcts2)

## Summary stats

In [ ]:
toc_parcels = df[df.num_TOC > 0]
non_toc_parcels = df[df.num_nonTOC > 0]
have_both_parcels = df[(df.num_TOC > 0) & (df.num_nonTOC > 0)]

print(f'# parcels: {len(df)}')
print(f'# parcels with TOC entitlements: {len(toc_parcels)}')
print(f'# parcels with non TOC entitlements: {len(non_toc_parcels)}')
print(f'# parcels with both TOC and non TOC entitlements: {len(have_both_parcels)}')
print(f'double check sum: {len(toc_parcels) + len(non_toc_parcels) - len(have_both_parcels)}')

In [ ]:
print(f'% parcels with TOC entitlements: {len(toc_parcels) / len(df)}')
print(f'% parcels with non TOC entitlements: {len(non_toc_parcels) / len(df)}')
print(f'% parcels with both entitlements: {len(have_both_parcels) / len(df)}')

In [ ]:
toc_parcels.zone_class.value_counts()

In [ ]:
non_toc_parcels.zone_class.value_counts()

In [ ]:
df.to_file(driver = 'GeoJSON', filename = '../gis/intermediate/toc_eligible_parcels_with_entitlements.geojson')

s3.upload_file('../gis/intermediate/toc_eligible_parcels_with_entitlements.geojson', 
               f'{bucket_name}', 'gis/intermediate/toc_eligible_parcels_with_entitlements.geojson')

## Breakdown by TOC Tiers

In [ ]:
def summarize_by_tiers(df):
    df2 = df.groupby('TOC_Tier').agg({'AIN':'count', 'num_TOC':'sum', 'num_nonTOC':'sum'}).reset_index()
    
    for i in ['TOC', 'nonTOC']:
        new_col = f'pct_{i}'
        numerator = f'num_{i}'
        df2[new_col] = df2[numerator] / (df2.num_TOC + df2.num_nonTOC)
    
    df2['all_AIN'] = df2.AIN.sum()
    df2['pct_AIN'] = df2.AIN / df2.all_AIN
    
    return df2

by_tiers = summarize_by_tiers(df)
by_tiers

## Breakdown by Zone Class

In [ ]:
def summarize_by_zones(df):
    df2 = df.groupby('zone_class').agg({'AIN':'count', 'num_TOC':'sum', 'num_nonTOC':'sum'}).reset_index()
    
    for i in ['TOC', 'nonTOC']:
        new_col = f'pct_{i}'
        numerator = f'num_{i}'
        df2[new_col] = df2[numerator] / (df2.num_TOC + df2.num_nonTOC)
    
    df2['all_AIN'] = df2.AIN.sum()
    df2['pct_AIN'] = df2.AIN / df2.all_AIN
    
    return df2

by_zones = summarize_by_zones(df)
by_zones

In [ ]:
writer = pd.ExcelWriter('../outputs/toc_charts.xlsx', engine = 'xlsxwriter')

by_tiers.to_excel(writer, sheet_name = 'entitlements_by_tier')
by_zones.to_excel(writer, sheet_name = 'entitlements_by_zone')

writer.save()